In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import seaborn as sns
from sklearn.manifold import TSNE


import kinematics
import FK_data_generator
import visualization
import network

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# load data 
data_path = "./data/joint_data_1000_2048.npy"
joint_clouds = np.load(data_path)  # [num_users, num_points_per_user, 4]
print(joint_clouds.shape) 

# LATENT_DIM = 32
# model = network.PointVAE(latent_dim=LATENT_DIM, num_points_k=1024).to(device)
# model.load_state_dict(torch.load('point_vae_model.pth', map_location=device))
# model.eval()
# model

In [ ]:
N = joint_clouds.shape[0]
K = joint_clouds.shape[1]

limits = {
    'x': (-0.7, 0.7),
    'y': (-0.7, 0.7),
    'z': (-0.7, 0.7)}

point_clouds = kinematics.forward_kinematics_pytorch(torch.tensor(joint_clouds).view(-1, 4))  # [N*K, 3]
point_clouds = point_clouds.view(N, K, 3).cpu().numpy()

In [ ]:
visualization.plot_N_joint_pairplots(joint_clouds[:10], marker_size=4)

In [ ]:
visualization.plot_N_task_point_cloud(point_clouds[:10], limits, 2, 3)

In [ ]:
visualization.plot_N_task_2d_slices(point_clouds[:10], limits, marker_size=3)

In [ ]:
# --- Ramdonly sample 5 latent vectors and decode ---
# TODO: match training set ones
# TODO: interpolation
# TODO: latent space arithmetic
with torch.no_grad():
    z = torch.randn(5, LATENT_DIM).to(device)  # [5, latent_dim]
    sampled_joint_cloud = model.decoder(z)  # [5, K, 4]
    sampled_joint_cloud = sampled_joint_cloud.cpu().numpy()

visualization.plot_N_joint_pairplots(sampled_joint_cloud, marker_size=5)

point_clouds_sampled = kinematics.forward_kinematics_pytorch(torch.tensor(sampled_joint_cloud).view(-1, 4))  # [N*K, 3]
point_clouds_sampled = point_clouds_sampled.view(5, K, 3).cpu().numpy()
visualization.plot_N_task_point_cloud(point_clouds_sampled, limits, 1, 5)

with torch.no_grad():
    recon_joint_cloud, _, _ = model(torch.tensor(sampled_joint_cloud).to(device))  # [N, K, 4]
    recon_joint_cloud = recon_joint_cloud.cpu().numpy()

visualization.plot_N_joint_pairplots(recon_joint_cloud, marker_size=5)
recon_point_clouds = kinematics.forward_kinematics_pytorch(torch.tensor(recon_joint_cloud).view(-1, 4))  # [N*K, 3]
recon_point_clouds = recon_point_clouds.view(5, K, 3).cpu().numpy()
visualization.plot_N_pair_comparison_task_point_cloud(point_clouds_sampled, recon_point_clouds, limits, marker_size=3)

In [ ]:
# --- test is latent space is smooth
with torch.no_grad():
    z = model.encoder(torch.tensor(joint_clouds[:1000]).to(device).float())  # [N, latent_dim]
    mu = model.fc_mu(z)
    logvar = model.fc_logvar(z)
    z = model.reparameterize(mu, logvar)
    z = z.cpu().numpy()
    
tsne = TSNE(n_components=2, random_state=42)
z_2d = tsne.fit_transform(z)  # [N, 2]
plt.figure(figsize=(8, 6))
plt.scatter(z_2d[:, 0], z_2d[:, 1], s=5)
plt.title("t-SNE Visualization of Latent Space")

# TODO: pick out a joint, varing limits, see clustering